In [62]:
import pickle
import pandas as pd
import numpy as np
import detectda as dtda
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sys
from joblib import Parallel, delayed
from scipy.stats import mode
import sys
sys.path.append("..")

In [63]:
import bclr

In [64]:
file=open('experiment1_data.pickle', 'rb')
exp1=pickle.load(file)

In [65]:
plt.rcParams["font.family"] = "Nunito"

In [66]:
kcp_tda_all = exp1['kcp_tda']
kcp_pca_all = exp1['kcp_pca']

cf_tda_all = exp1['cf_tda']
cf_pca_all = exp1['cf_pca']

bclr_tda_all = exp1['bcc_tda']
bclr_pca_all = exp1['bcc_pca']

### bclr changepoint detection

In [67]:
cp_bclr_tda = [mode(bclr.k_draws_[2500:], keepdims=False).mode  for bclr in bclr_tda_all]
cp_bclr_pca = [mode(bclr.k_draws_[2500:], keepdims=False).mode  for bclr in bclr_pca_all]

In [68]:
def rmse(x):
    return np.sqrt(np.mean(x**2))

cp_bclr_mses_tda = [rmse(bclr.post_k-25)  for bclr in bclr_tda_all]
cp_bclr_mses_pca = [rmse(bclr.post_k-25)  for bclr in bclr_pca_all]

In [69]:
rmse(np.array(cp_bclr_pca)-25)

4.275160815688692

In [70]:
cp_bclr_tda_mean = [np.mean(bclr.k_draws_[2500:]) for bclr in bclr_tda_all]
cp_bclr_pca_mean = [np.mean(bclr.k_draws_[2500:]) for bclr in bclr_pca_all]

In [71]:
rmse(np.array(cp_bclr_tda_mean)-25)
rmse(np.array(cp_bclr_pca_mean)-25)

3.901440176181098

In [72]:
print("Mean RMSE: %0.3f" % np.mean(cp_bclr_mses_tda))
print("SE RMSE: %0.3f" % np.std(cp_bclr_mses_tda))

Mean RMSE: 0.948
SE RMSE: 0.783


In [73]:
print("Mean RMSE: %0.3f" % np.mean(cp_bclr_mses_pca))
print("SE RMSE: %0.3f" % np.std(cp_bclr_mses_pca))

Mean RMSE: 3.849
SE RMSE: 2.158


In [74]:
print("RMSE for bclr, TDA features: %0.3f" % np.sqrt(np.mean((np.array(cp_bclr_tda)-25)**2)))
print("RMSE for bclr, PCA features: %0.3f" % np.sqrt(np.mean((np.array(cp_bclr_pca)-25)**2)))

print("Probability k=25 for bclr, TDA features: %0.3f" % np.mean(np.array(cp_bclr_tda)==25))
print("Probability k=25 for bclr, PCA features: %0.3f" % np.mean(np.array(cp_bclr_pca)==25))

print("Std Err: Probability k=25 for bclr, TDA features: %0.3f" % (np.std(np.array(cp_bclr_tda)==25)/np.sqrt(1000)))
print("Std Err: Probability k=25 for bclr, PCA features: %0.3f" % (np.std(np.array(cp_bclr_pca)==25)/np.sqrt(1000)))

RMSE for bclr, TDA features: 1.072
RMSE for bclr, PCA features: 4.275
Probability k=25 for bclr, TDA features: 0.697
Probability k=25 for bclr, PCA features: 0.096
Std Err: Probability k=25 for bclr, TDA features: 0.015
Std Err: Probability k=25 for bclr, PCA features: 0.009


In [75]:
def hpm(arr, alpha=0.05):
    probs = pd.DataFrame(arr).value_counts(normalize=True)
    ind = np.where(probs.cumsum() > 1-alpha)[0][0]
    high_mass = np.array([i[0] for i in probs.index[:(ind+1)]])
    return high_mass

In [100]:
arr = np.array([25, 25, 25, 25, 25, 26, 26, 26, 26, 26])
hpm(arr, 0.51)

array([25])

In [82]:
for al in [0.5, 0.2, 0.1, 0.05, 0.01]:    
    m2_bclr_tda = [bool(np.isin(25, hpm(bclr_obj.k_draws_[2500:], al))) for bclr_obj in bclr_tda_all] 
    m2_bclr_pca = [bool(np.isin(25, hpm(bclr_obj.k_draws_[2500:], al))) for bclr_obj in bclr_pca_all]
                  
    print(1-al, np.mean(m2_bclr_tda), "tda hpm")
    print(1-al, np.mean(m2_bclr_pca), "pca hpm")

[25.]
0.5 0.749 tda hpm
0.5 0.168 pca hpm
[25. 26.]
0.8 0.887 tda hpm
0.8 0.32 pca hpm
[25. 26. 27.]
0.9 0.939 tda hpm
0.9 0.432 pca hpm
[25. 26. 27.]
0.95 0.962 tda hpm
0.95 0.511 pca hpm
[25. 26. 27.]
0.99 0.991 tda hpm
0.99 0.659 pca hpm


In [25]:
for al in [0.25, 0.1, 0.05, 0.025, 0.005]:    
    m1_bclr_tda = [(np.quantile(bclr.k_draws_[2500:], al), np.quantile(bclr.k_draws_[2500:], 1-al)) for bclr in bclr_tda_all] 
    m1_bclr_pca = [(np.quantile(bclr.k_draws_[2500:], al), np.quantile(bclr.k_draws_[2500:], 1-al)) for bclr in bclr_pca_all]

    print(1-al*2, np.mean([x[1] >= 25 and x[0] <= 25 for x in m1_bclr_tda]), "tda")
    print(1-al*2, np.mean([x[1] >= 25 and x[0] <= 25 for x in m1_bclr_pca]), "pca")

0.5 0.852 tda
0.5 0.312 pca
0.8 0.933 tda
0.8 0.509 pca
0.9 0.964 tda
0.9 0.594 pca
0.95 0.972 tda
0.95 0.645 pca
0.99 0.994 tda
0.99 0.721 pca


In [26]:
np.mean([x[1] >= 25 and x[0] <= 25 for x in m1_bclr_tda])

0.994

In [27]:
np.mean([x[1] >= 25 and x[0] <= 25 for x in m1_bclr_pca])

0.721

Let's look at the beta coefficients

In [28]:
cp_bclr_tda_beta = np.array([bclr.beta_draws_[2500:] for bclr in bclr_tda_all])

In [29]:
names = ['mean_0_mid', 'variance_0_mid', 'skewness_0_mid', 'kurtosis_0_mid', 'median_0_mid', 'iqr_0_mid', 'q25_0_mid', 'q75_0_mid', 'pers_entr_0_life', 'alps_0_life', 'mean_0_life', 'variance_0_life','skewness_0_life', 'kurtosis_0_life', 'median_0_life', 'iqr_0_life', 'q25_0_life', 'q75_0_life', 'mean_1_mid', 'variance_1_mid', 'skewness_1_mid', 'kurtosis_1_mid', 'median_1_mid', 'iqr_1_mid','q25_1_mid', 'q75_1_mid', 'pers_entr_1_life', 'alps_1_life','mean_1_life', 'variance_1_life', 'skewness_1_life', 'kurtosis_1_life','median_1_life', 'iqr_1_life', 'q25_1_life', 'q75_1_life']
beta_cors = np.array([np.corrcoef(bcbeta, rowvar=False) for bcbeta in cp_bclr_tda_beta])
bcors = np.mean(beta_cors, axis=0)

In [30]:
nam1 = []
nam2 = []
cors = []
for i in range(35):
    for j in range(i+1, 36):
        nam1.append(names[i])
        nam2.append(names[j])
        cors.append(bcors[i,j])

In [31]:
cor_df = pd.DataFrame({'Var1': nam1, 'Var2': nam2, 'Corr': cors})
cor_df.reindex(cor_df['Corr'].abs().sort_values(ascending=False).index)[:20]

,Var1,Var2,Corr
615,skewness_1_life,kurtosis_1_life,-0.433883
421,iqr_0_life,q75_0_life,-0.429776
628,iqr_1_life,q75_1_life,-0.367770
354,skewness_0_life,kurtosis_0_life,-0.358401
166,iqr_0_mid,q75_0_mid,-0.336076
595,alps_1_life,variance_1_life,-0.325158
165,iqr_0_mid,q25_0_mid,0.305397
552,iqr_1_mid,q25_1_mid,0.303774
553,iqr_1_mid,q75_1_mid,-0.281507
587,pers_entr_1_life,variance_1_life,0.259092


In [32]:
beta_snr = np.array([np.mean(bcbeta, axis=0)**2/np.var(bcbeta, axis=0) for bcbeta in cp_bclr_tda_beta])

In [33]:
abc = pd.Series(np.array(names)[np.argmax(beta_snr, axis=1)]).value_counts(normalize=True)
prop_largest = abc.reindex(names).fillna(0)
df_prop = pd.DataFrame({'names': names, 'mean_snr': np.mean(beta_snr, axis=0), 'sd_snr': np.std(beta_snr, axis=0),
              'prop_highest': prop_largest}).sort_values("prop_highest", ascending=False)

In [34]:
nam_5 = list(df_prop.head(5).index)
nam_list = np.array(names)

In [35]:
cor_index = [np.where(nam_list == nam)[0][0] for nam in nam_5]

In [36]:
for i in cor_index:
    for j in cor_index:
        if i<j:
            print(nam_list[i], "\t\t", nam_list[j], "\t\t", np.round(bcors[i,j],3))

pers_entr_0_life 		 skewness_0_life 		 0.079
pers_entr_0_life 		 kurtosis_0_life 		 0.043
pers_entr_0_life 		 alps_0_life 		 -0.007
pers_entr_0_life 		 variance_0_life 		 0.2
skewness_0_life 		 kurtosis_0_life 		 -0.358
alps_0_life 		 skewness_0_life 		 0.12
alps_0_life 		 kurtosis_0_life 		 0.149
alps_0_life 		 variance_0_life 		 -0.112
variance_0_life 		 skewness_0_life 		 -0.128
variance_0_life 		 kurtosis_0_life 		 -0.119


### Changeforest changepoint detection

In [38]:
cp_cf_tda = [a['best_split'] for a in cf_tda_all]
cp_cf_pca = [a['best_split'] for a in cf_pca_all]

print("RMSE for CF, TDA features: %0.3f" % np.sqrt(np.mean((np.array(cp_cf_tda)-25)**2)))
print("RMSE for CF, PCA features: %0.3f" % np.sqrt(np.mean((np.array(cp_cf_pca)-25)**2)))

RMSE for CF, TDA features: 1.021
RMSE for CF, PCA features: 16.998


In [41]:
print("Probability k=25 for CF, TDA features: %0.3f" % np.mean(np.array(cp_cf_tda)==25))
print("Probability k=25 for CF, PCA features: %0.3f" % np.mean(np.array(cp_cf_pca)==25))

print("Std Err: Probability k=25 for CF, TDA features: %0.3f" % (np.std(np.array(cp_cf_tda)==25)/np.sqrt(1000)))
print("Std Err: Probability k=25 for CF, PCA features: %0.3f" % (np.std(np.array(cp_cf_pca)==25)/np.sqrt(1000)))

Probability k=25 for CF, TDA features: 0.714
Probability k=25 for CF, PCA features: 0.020
Std Err: Probability k=25 for CF, TDA features: 0.014
Std Err: Probability k=25 for CF, PCA features: 0.004


Kernel Changepoint detection using <code>ruptures</code> package

In [42]:
cp_kcp_tda = [a['changepoint'][0] for a in kcp_tda_all]
cp_kcp_pca = [a['changepoint'][0] for a in kcp_pca_all]

Changepoint distribution for Kernel Changepoint detection and dynamic programming using TDA features

In [44]:
print("RMSE for KCP, TDA features: %0.3f" % np.sqrt(np.mean((np.array(cp_kcp_tda)-25)**2)))
print("RMSE for KCP, PCA features: %0.3f" % np.sqrt(np.mean((np.array(cp_kcp_pca)-25)**2)))

RMSE for KCP, TDA features: 1.382
RMSE for KCP, PCA features: 15.365


In [49]:
print("Probability k=25 for KCP, TDA features: %0.3f" % np.mean(np.array(cp_kcp_tda)==25))
print("Probability k=25 for KCP, PCA features: %0.3f" % np.mean(np.array(cp_kcp_pca)==25))

print("Std Err: Probability k=25 for KCP, TDA features: %0.3f" % (np.std(np.array(cp_kcp_tda)==25)/np.sqrt(1000)))
print("Std Err: Probability k=25 for KCP, PCA features: %0.3f" % (np.std(np.array(cp_kcp_pca)==25)/np.sqrt(1000)))

Probability k=25 for KCP, TDA features: 0.673
Probability k=25 for KCP, PCA features: 0.009
Std Err: Probability k=25 for KCP, TDA features: 0.015
Std Err: Probability k=25 for KCP, PCA features: 0.003


In [30]:
pd.Series(cp_kcp_tda).value_counts(normalize=True).sort_index()

20    0.001
21    0.001
22    0.010
23    0.013
24    0.055
25    0.673
26    0.145
27    0.058
28    0.018
29    0.007
30    0.005
31    0.005
32    0.001
33    0.005
34    0.001
36    0.001
37    0.001
dtype: float64

Changepoint distribution for Kernel Changepoint detection and dynamic programming using PCA features

In [31]:
pd.Series(cp_kcp_pca).value_counts(normalize=True).sort_index()

2     0.037
3     0.038
4     0.037
5     0.032
6     0.022
7     0.024
8     0.019
9     0.021
10    0.020
11    0.021
12    0.019
13    0.022
14    0.019
15    0.015
16    0.011
17    0.013
18    0.019
19    0.014
20    0.017
21    0.013
22    0.013
23    0.019
24    0.013
25    0.009
26    0.016
27    0.020
28    0.017
29    0.016
30    0.014
31    0.021
32    0.015
33    0.017
34    0.024
35    0.023
36    0.019
37    0.027
38    0.016
39    0.021
40    0.024
41    0.015
42    0.019
43    0.015
44    0.026
45    0.025
46    0.037
47    0.041
48    0.045
dtype: float64

Import experiment 1 data

In [3]:
ims = np.load("Experiment1_Minus2_Images1000.npy")

In [4]:
noise_data = np.reshape(ims, (50, -1))

In [9]:
noisePCA = PCA(n_components=36).fit_transform(noise_data)